In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import json
DataSet = "F:/TCP007/data/model_input/" # 数据集

In [2]:
def load_data(filename):
    """获取数据"""
    arr=np.load(filename,allow_pickle=True)
    arr1=[]
    arr2=[]
    for i in range(len(arr)):
        arr1.append(arr[i][0][0:20])
        if arr[i][1]>=15:
            arr2.append(15)
        else:
            arr2.append(arr[i][1])
    return arr1,arr2,len(arr1)
    
class GetData:
    """打乱数据集"""
    def __init__(self, filenames, need_shuffle):  # 训练集需要打乱
        all_data = []
        all_labels = []
        '''for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)'''
        data,labels,data_size=load_data(filenames)
        all_data.append(data)
        all_labels.append(labels)
        self._size=data_size
        self._data = np.vstack(all_data)  # 转为纵向矩阵,10000组，每组3072数
        self._data = np.reshape(self._data,(len(self._data),-1))  # 转为纵向矩阵,10000组，每组3072数
        # print(self._data)
        self._labels = np.hstack(all_labels)  # 转为横向矩阵，10000个数
        # print(self._data.shape)
        # print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]  # 训练集总数量
        # print(self._num_examples)
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前遍历数据集的位置
        if self._need_shuffle:  # 判断是否需要打乱数据
            self._shuffle_data()
    def getSize(self):
        return self._size
    
    def clear(self):
        self._indicator=0

    def _shuffle_data(self):  # 打乱数据
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):  # 数据分组，每次取不同的组
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:  # 考察位置大于总数，重新打乱数据，重新分组
            if self._need_shuffle:  # 可以打乱
                self._shuffle_data()  # 重新打乱
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:  # 分块大小过大
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
# train_data=GetData("train_file_1.npy",True)
test_data=GetData(DataSet+"valid_file.npy",False)

In [3]:
def conv_wrapper(inputs,name,output_channel=32,\
    kernel_size=(3,3),strides=1,activation=tf.nn.relu):
    """卷积层装饰器"""
    # with batch normalization:conv->bn->activation
    with tf.name_scope(name):
        conv2d=tf.layers.conv2d(inputs,
                         output_channel,
                         kernel_size,
                         strides=strides,
                         padding='same',
                         activation=None,
                         trainable=False,
                         data_format='channels_first',
                         name=name+'/conv2d')
        bn=tf.layers.batch_normalization(conv2d, training=False,trainable=False)
        return activation(bn)
        
def pooling_wrapper(inputs,name):
    """池化层装饰器"""
    return tf.layers.max_pooling2d(inputs,(2,2),(2,2),name=name,padding='same',data_format='channels_first')


In [4]:
# 定义计算图
x = tf.placeholder(tf.float32, [None, 2560])  # 设置占位符
x_image=tf.reshape(x,[-1,1,20,128])  # 图像规模
x_image=tf.transpose(x_image,perm=[0,2,3,1])  # 通道转换
y = tf.placeholder(tf.int64, [None])  # y为的标注

#conv1
conv1_1=conv_wrapper(x_image,'conv1_1')
conv1_2=conv_wrapper(conv1_1,'conv1_2')
conv1_3=conv_wrapper(conv1_2,'conv1_3')
pooling1=pooling_wrapper(conv1_3,'pool1')
#conv2
conv2_1=conv_wrapper(pooling1,'conv2_1',output_channel=64)
conv2_2=conv_wrapper(conv2_1,'conv2_2',output_channel=64)
conv2_3=conv_wrapper(conv2_2,'conv2_3',output_channel=64)
pooling2=pooling_wrapper(conv2_3,'pool2')
#conv3
conv3_1=conv_wrapper(pooling2,'conv3_1',output_channel=128)
conv3_2=conv_wrapper(conv3_1,'conv3_2',output_channel=128)
conv3_3=conv_wrapper(conv3_2,'conv3_3',output_channel=128)
pooling3=pooling_wrapper(conv3_3,'pool3')

#flat(平坦化)
flatten=tf.layers.flatten(pooling3)  # 在保留第0轴的情况下对输入的张量进行Flatten(扁平化)
#输出 全连接层 输出形状[?,10]
logits=tf.layers.dense(flatten,16)  # 全连接层

loss=tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)  # 交叉熵损失函数：y_->softmax,y->onhot,loss=ylogy_

predict = tf.argmax(logits,1)  # 样本中分布的最大值的位置，得到index

correct_prediction = tf.equal(predict, y)  # 相等为1，不相等为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))  # 平均数即为准确率

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)  # adam优化器，定义梯度下降方法，使损失函数最小


In [5]:
# 执行计算图
correct_result=[[] for x in range(16)]
predict_result=[[] for x in range(16)]
protocol_cnt=[0 for x in range(16)]
ans=[]
saver = tf.train.Saver()
test_data.clear()
with tf.Session() as sess:
    init = tf.global_variables_initializer()  # 变量初始化
    sess.run(init)
    saver.restore(sess, "./Model/VGGmodel2/VGG11999.ckpt")
    corr_cnt=0
    fp=open('output1.txt','a')
    for i in range(test_data.getSize()):
        datas,labels=test_data.next_batch(1)
        pre_val = sess.run(
            predict,
            feed_dict={
                x: datas,
                y: labels})
        isRight=0
        if labels[0]==pre_val[0]:
            isRight=1
        ans.append({'id':i,'correct':labels[0],'predict':pre_val[0],'isRight':isRight})
        print('ID:%d correct_label:%d predict_label:%d isRight:%d' % (i,labels[0],pre_val[0],isRight))
        fp.write('ID:%d correct_label:%d predict_label:%d isRight:%d' % (i,labels[0],pre_val[0],isRight))
        fp.write('\n')
        protocol_cnt[labels[0]]+=1
        correct_result[labels[0]].append([i,pre_val[0]])
        predict_result[pre_val[0]].append([i,labels[0]])
        if labels[0]==pre_val:
            corr_cnt=corr_cnt+1
    print(corr_cnt)
    fp.write(str(corr_cnt))
    fp.close()

INFO:tensorflow:Restoring parameters from ./Model/VGGmodel2/VGG11999.ckpt
ID:0 correct_label:0 predict_label:0 isRight:1
ID:1 correct_label:0 predict_label:0 isRight:1
ID:2 correct_label:0 predict_label:0 isRight:1
ID:3 correct_label:0 predict_label:3 isRight:0
ID:4 correct_label:0 predict_label:0 isRight:1
ID:5 correct_label:0 predict_label:0 isRight:1
ID:6 correct_label:0 predict_label:8 isRight:0
ID:7 correct_label:0 predict_label:3 isRight:0
ID:8 correct_label:0 predict_label:8 isRight:0
ID:9 correct_label:0 predict_label:5 isRight:0
ID:10 correct_label:0 predict_label:3 isRight:0
ID:11 correct_label:0 predict_label:0 isRight:1
ID:12 correct_label:0 predict_label:3 isRight:0
ID:13 correct_label:0 predict_label:0 isRight:1
ID:14 correct_label:0 predict_label:5 isRight:0
ID:15 correct_label:0 predict_label:3 isRight:0
ID:16 correct_label:0 predict_label:2 isRight:0
ID:17 correct_label:0 predict_label:0 isRight:1
ID:18 correct_label:0 predict_label:3 isRight:0
ID:19 correct_label:0 pr

ID:177 correct_label:0 predict_label:0 isRight:1
ID:178 correct_label:0 predict_label:4 isRight:0
ID:179 correct_label:0 predict_label:10 isRight:0
ID:180 correct_label:0 predict_label:3 isRight:0
ID:181 correct_label:0 predict_label:0 isRight:1
ID:182 correct_label:0 predict_label:4 isRight:0
ID:183 correct_label:0 predict_label:3 isRight:0
ID:184 correct_label:0 predict_label:0 isRight:1
ID:185 correct_label:0 predict_label:10 isRight:0
ID:186 correct_label:0 predict_label:3 isRight:0
ID:187 correct_label:0 predict_label:10 isRight:0
ID:188 correct_label:0 predict_label:0 isRight:1
ID:189 correct_label:0 predict_label:5 isRight:0
ID:190 correct_label:0 predict_label:5 isRight:0
ID:191 correct_label:0 predict_label:0 isRight:1
ID:192 correct_label:0 predict_label:8 isRight:0
ID:193 correct_label:0 predict_label:0 isRight:1
ID:194 correct_label:0 predict_label:3 isRight:0
ID:195 correct_label:0 predict_label:5 isRight:0
ID:196 correct_label:0 predict_label:15 isRight:0
ID:197 correct_l

ID:365 correct_label:0 predict_label:4 isRight:0
ID:366 correct_label:0 predict_label:4 isRight:0
ID:367 correct_label:0 predict_label:4 isRight:0
ID:368 correct_label:0 predict_label:0 isRight:1
ID:369 correct_label:0 predict_label:3 isRight:0
ID:370 correct_label:0 predict_label:3 isRight:0
ID:371 correct_label:0 predict_label:0 isRight:1
ID:372 correct_label:0 predict_label:0 isRight:1
ID:373 correct_label:0 predict_label:4 isRight:0
ID:374 correct_label:0 predict_label:4 isRight:0
ID:375 correct_label:0 predict_label:4 isRight:0
ID:376 correct_label:0 predict_label:0 isRight:1
ID:377 correct_label:0 predict_label:0 isRight:1
ID:378 correct_label:0 predict_label:0 isRight:1
ID:379 correct_label:0 predict_label:0 isRight:1
ID:380 correct_label:0 predict_label:2 isRight:0
ID:381 correct_label:0 predict_label:2 isRight:0
ID:382 correct_label:0 predict_label:0 isRight:1
ID:383 correct_label:0 predict_label:0 isRight:1
ID:384 correct_label:0 predict_label:4 isRight:0
ID:385 correct_label

ID:555 correct_label:0 predict_label:4 isRight:0
ID:556 correct_label:0 predict_label:4 isRight:0
ID:557 correct_label:0 predict_label:4 isRight:0
ID:558 correct_label:0 predict_label:4 isRight:0
ID:559 correct_label:0 predict_label:3 isRight:0
ID:560 correct_label:0 predict_label:3 isRight:0
ID:561 correct_label:0 predict_label:3 isRight:0
ID:562 correct_label:0 predict_label:3 isRight:0
ID:563 correct_label:0 predict_label:4 isRight:0
ID:564 correct_label:0 predict_label:3 isRight:0
ID:565 correct_label:0 predict_label:0 isRight:1
ID:566 correct_label:0 predict_label:3 isRight:0
ID:567 correct_label:0 predict_label:0 isRight:1
ID:568 correct_label:0 predict_label:3 isRight:0
ID:569 correct_label:0 predict_label:3 isRight:0
ID:570 correct_label:0 predict_label:0 isRight:1
ID:571 correct_label:0 predict_label:0 isRight:1
ID:572 correct_label:0 predict_label:0 isRight:1
ID:573 correct_label:0 predict_label:0 isRight:1
ID:574 correct_label:0 predict_label:4 isRight:0
ID:575 correct_label

ID:742 correct_label:0 predict_label:10 isRight:0
ID:743 correct_label:0 predict_label:4 isRight:0
ID:744 correct_label:0 predict_label:10 isRight:0
ID:745 correct_label:0 predict_label:9 isRight:0
ID:746 correct_label:0 predict_label:7 isRight:0
ID:747 correct_label:0 predict_label:4 isRight:0
ID:748 correct_label:0 predict_label:2 isRight:0
ID:749 correct_label:0 predict_label:10 isRight:0
ID:750 correct_label:0 predict_label:10 isRight:0
ID:751 correct_label:0 predict_label:7 isRight:0
ID:752 correct_label:0 predict_label:7 isRight:0
ID:753 correct_label:0 predict_label:2 isRight:0
ID:754 correct_label:0 predict_label:10 isRight:0
ID:755 correct_label:0 predict_label:14 isRight:0
ID:756 correct_label:0 predict_label:10 isRight:0
ID:757 correct_label:0 predict_label:14 isRight:0
ID:758 correct_label:0 predict_label:15 isRight:0
ID:759 correct_label:0 predict_label:10 isRight:0
ID:760 correct_label:0 predict_label:10 isRight:0
ID:761 correct_label:0 predict_label:7 isRight:0
ID:762 co

...

ID:11742 correct_label:15 predict_label:5 isRight:0
ID:11743 correct_label:15 predict_label:5 isRight:0
ID:11744 correct_label:15 predict_label:5 isRight:0
ID:11745 correct_label:15 predict_label:5 isRight:0
ID:11746 correct_label:15 predict_label:5 isRight:0
ID:11747 correct_label:15 predict_label:5 isRight:0
ID:11748 correct_label:15 predict_label:5 isRight:0
ID:11749 correct_label:15 predict_label:5 isRight:0
ID:11750 correct_label:15 predict_label:5 isRight:0
ID:11751 correct_label:15 predict_label:5 isRight:0
ID:11752 correct_label:15 predict_label:5 isRight:0
ID:11753 correct_label:15 predict_label:5 isRight:0
ID:11754 correct_label:15 predict_label:5 isRight:0
ID:11755 correct_label:15 predict_label:5 isRight:0
ID:11756 correct_label:15 predict_label:5 isRight:0
ID:11757 correct_label:15 predict_label:5 isRight:0
ID:11758 correct_label:15 predict_label:5 isRight:0
ID:11759 correct_label:15 predict_label:5 isRight:0
ID:11760 correct_label:15 predict_label:5 isRight:0
ID:11761 cor

ID:11901 correct_label:15 predict_label:7 isRight:0
ID:11902 correct_label:15 predict_label:7 isRight:0
ID:11903 correct_label:15 predict_label:7 isRight:0
ID:11904 correct_label:15 predict_label:7 isRight:0
ID:11905 correct_label:15 predict_label:7 isRight:0
ID:11906 correct_label:15 predict_label:7 isRight:0
ID:11907 correct_label:15 predict_label:7 isRight:0
ID:11908 correct_label:15 predict_label:7 isRight:0
ID:11909 correct_label:15 predict_label:7 isRight:0
ID:11910 correct_label:15 predict_label:7 isRight:0
ID:11911 correct_label:15 predict_label:10 isRight:0
ID:11912 correct_label:15 predict_label:10 isRight:0
ID:11913 correct_label:15 predict_label:10 isRight:0
ID:11914 correct_label:15 predict_label:10 isRight:0
ID:11915 correct_label:15 predict_label:10 isRight:0
ID:11916 correct_label:15 predict_label:10 isRight:0
ID:11917 correct_label:15 predict_label:11 isRight:0
ID:11918 correct_label:15 predict_label:11 isRight:0
ID:11919 correct_label:15 predict_label:11 isRight:0
ID:

ID:12073 correct_label:15 predict_label:14 isRight:0
ID:12074 correct_label:15 predict_label:14 isRight:0
ID:12075 correct_label:15 predict_label:14 isRight:0
ID:12076 correct_label:15 predict_label:2 isRight:0
ID:12077 correct_label:15 predict_label:10 isRight:0
ID:12078 correct_label:15 predict_label:10 isRight:0
ID:12079 correct_label:15 predict_label:2 isRight:0
ID:12080 correct_label:15 predict_label:2 isRight:0
ID:12081 correct_label:15 predict_label:10 isRight:0
ID:12082 correct_label:15 predict_label:2 isRight:0
ID:12083 correct_label:15 predict_label:7 isRight:0
ID:12084 correct_label:15 predict_label:8 isRight:0
ID:12085 correct_label:15 predict_label:15 isRight:1
ID:12086 correct_label:15 predict_label:7 isRight:0
ID:12087 correct_label:15 predict_label:8 isRight:0
ID:12088 correct_label:15 predict_label:15 isRight:1
ID:12089 correct_label:15 predict_label:15 isRight:1
ID:12090 correct_label:15 predict_label:15 isRight:1
ID:12091 correct_label:15 predict_label:15 isRight:1
I

ID:12234 correct_label:15 predict_label:10 isRight:0
ID:12235 correct_label:15 predict_label:2 isRight:0
ID:12236 correct_label:15 predict_label:10 isRight:0
ID:12237 correct_label:15 predict_label:2 isRight:0
ID:12238 correct_label:15 predict_label:10 isRight:0
ID:12239 correct_label:15 predict_label:2 isRight:0
ID:12240 correct_label:15 predict_label:10 isRight:0
ID:12241 correct_label:15 predict_label:10 isRight:0
ID:12242 correct_label:15 predict_label:10 isRight:0
ID:12243 correct_label:15 predict_label:2 isRight:0
ID:12244 correct_label:15 predict_label:10 isRight:0
ID:12245 correct_label:15 predict_label:2 isRight:0
ID:12246 correct_label:15 predict_label:10 isRight:0
ID:12247 correct_label:15 predict_label:2 isRight:0
ID:12248 correct_label:15 predict_label:14 isRight:0
ID:12249 correct_label:15 predict_label:2 isRight:0
ID:12250 correct_label:15 predict_label:10 isRight:0
ID:12251 correct_label:15 predict_label:15 isRight:1
ID:12252 correct_label:15 predict_label:15 isRight:1


ID:12418 correct_label:15 predict_label:15 isRight:1
ID:12419 correct_label:15 predict_label:15 isRight:1
ID:12420 correct_label:15 predict_label:15 isRight:1
ID:12421 correct_label:15 predict_label:15 isRight:1
ID:12422 correct_label:15 predict_label:15 isRight:1
ID:12423 correct_label:15 predict_label:15 isRight:1
ID:12424 correct_label:15 predict_label:15 isRight:1
ID:12425 correct_label:15 predict_label:15 isRight:1
ID:12426 correct_label:15 predict_label:15 isRight:1
ID:12427 correct_label:15 predict_label:15 isRight:1
ID:12428 correct_label:15 predict_label:15 isRight:1
ID:12429 correct_label:15 predict_label:15 isRight:1
ID:12430 correct_label:15 predict_label:15 isRight:1
ID:12431 correct_label:15 predict_label:15 isRight:1
ID:12432 correct_label:15 predict_label:8 isRight:0
ID:12433 correct_label:15 predict_label:10 isRight:0
ID:12434 correct_label:15 predict_label:10 isRight:0
ID:12435 correct_label:15 predict_label:2 isRight:0
ID:12436 correct_label:15 predict_label:2 isRigh

In [6]:
ans2=[]
dic=np.load("数据包/protocol_dictionary.npy",allow_pickle=True)[()]  # 协议字典
print("数据报文数目：%d" % test_data.getSize())
print("预测正确数目：%d" % corr_cnt)
print("整体准确率：%.4f" % (corr_cnt/test_data.getSize()))
ans2.append({'数据报文数目':test_data.getSize(),'预测正确数目':corr_cnt,'整体准确率':(corr_cnt/test_data.getSize())})
# 协议准确率、召回率等...
fp=open('output2.txt','a')
for i in range(16):
    TP=0
    FP=0
    FN=0
    for j in range(len(correct_result[i])):
        if(correct_result[i][j][1]==i or (correct_result[i][j][1]>=15 and i>=15)):  # 预测正确
            TP+=1
        elif(correct_result[i][j][1]!=i):  # 真预测为假
            FN+=1
    for j in range(len(predict_result[i])):
        if((predict_result[i][j][1]!=i and i<16) or (i>=16 and predict_result[i][j][1]<16)):  # 假预测为真
            FP+=1
    if TP==0:
        print("%-8s \tAll:%5d    TP:    0    FN:%5d    FP:%5d    召回率:    0     精准度:    0     f1值:    0" \
              % (list(dic.keys())[i],protocol_cnt[i],FN,FP))
        fp.write("%-8s \tAll:%5d    TP:    0    FN:%5d    FP:%5d    召回率:    0     精准度:    0     f1值:    0\n" \
              % (list(dic.keys())[i],protocol_cnt[i],FN,FP))
        ans2.append({'protocol':list(dic.keys())[i],'All':protocol_cnt[i],'TP':0,'FN':FN,'FP':FP,'召回率':0,'精准度':0,'f1值':0})
        continue    
    recall=TP/(TP+FN)
    prec=TP/(TP+FP)
    f1=2*recall*prec/(recall+prec)
    print("%-8s \tAll:%5d    TP:%5d    FN:%5d    FP:%5d    召回率:%.4f    精准度:%.4f    f1值:%.4f" \
          % (list(dic.keys())[i],protocol_cnt[i],TP,FN,FP,recall,prec,f1)) 
    fp.write("%-8s \tAll:%5d    TP:%5d    FN:%5d    FP:%5d    召回率:%.4f    精准度:%.4f    f1值:%.4f\n" \
          % (list(dic.keys())[i],protocol_cnt[i],TP,FN,FP,recall,prec,f1))
    ans2.append({'protocol':list(dic.keys())[i],'All':protocol_cnt[i],'TP':TP,'FN':FN,'FP':FP,'召回率':recall,'精准度':prec,'f1值':f1})
fp.close()

数据报文数目：12493
预测正确数目：8043
整体准确率：0.6438
UDP      	All: 1050    TP:  243    FN:  807    FP:   61    召回率:0.2314    精准度:0.7993    f1值:0.3589
TCP      	All: 1050    TP:  371    FN:  679    FP:   60    召回率:0.3533    精准度:0.8608    f1值:0.5010
TLSv1.2  	All: 1050    TP:  554    FN:  496    FP:  330    召回率:0.5276    精准度:0.6267    f1值:0.5729
ARP      	All: 1050    TP: 1050    FN:    0    FP:  845    召回率:1.0000    精准度:0.5541    f1值:0.7131
STUN     	All: 1050    TP:  966    FN:   84    FP:  532    召回率:0.9200    精准度:0.6449    f1值:0.7582
LLMNR    	All: 1050    TP: 1048    FN:    2    FP:  206    召回率:0.9981    精准度:0.8357    f1值:0.9097
NBNS     	All: 1050    TP: 1049    FN:    1    FP:   60    召回率:0.9990    精准度:0.9459    f1值:0.9717
SNMP     	All: 1050    TP:  806    FN:  244    FP:  512    召回率:0.7676    精准度:0.6115    f1值:0.6807
DNS      	All: 1050    TP:  741    FN:  309    FP:  152    召回率:0.7057    精准度:0.8298    f1值:0.7627
MDNS     	All:  909    TP:  549    FN:  360    FP:  421    召回率:0.6040    精准度:0.5

In [7]:
ans2

[{'数据报文数目': 12493, '预测正确数目': 8043, '整体准确率': 0.6438005282958457},
 {'protocol': 'UDP',
  'All': 1050,
  'TP': 243,
  'FN': 807,
  'FP': 61,
  '召回率': 0.23142857142857143,
  '精准度': 0.7993421052631579,
  'f1值': 0.35893648449039883},
 {'protocol': 'TCP',
  'All': 1050,
  'TP': 371,
  'FN': 679,
  'FP': 60,
  '召回率': 0.35333333333333333,
  '精准度': 0.8607888631090487,
  'f1值': 0.5010128291694801},
 {'protocol': 'TLSv1.2',
  'All': 1050,
  'TP': 554,
  'FN': 496,
  'FP': 330,
  '召回率': 0.5276190476190477,
  '精准度': 0.6266968325791855,
  'f1值': 0.5729058945191313},
 {'protocol': 'ARP',
  'All': 1050,
  'TP': 1050,
  'FN': 0,
  'FP': 845,
  '召回率': 1.0,
  '精准度': 0.554089709762533,
  'f1值': 0.7130730050933787},
 {'protocol': 'STUN',
  'All': 1050,
  'TP': 966,
  'FN': 84,
  'FP': 532,
  '召回率': 0.92,
  '精准度': 0.6448598130841121,
  'f1值': 0.7582417582417582},
 {'protocol': 'LLMNR',
  'All': 1050,
  'TP': 1048,
  'FN': 2,
  'FP': 206,
  '召回率': 0.9980952380952381,
  '精准度': 0.835725677830941,
  'f1值': 0.90

In [8]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
        
with open('test_label.json','w') as f:
    json.dump(ans,f,cls=NpEncoder)
with open('test_res.json','w') as f:
    json.dump(ans2,f,cls=NpEncoder)